<a href="https://colab.research.google.com/github/gsc16/MachineLearning/blob/master/hw1q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [1]:
import keras
import numpy as np
from keras.datasets import mnist
import math
import matplotlib as plt

Using TensorFlow backend.


Loading images from MNIST dataset

In [0]:
(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

Displaying the original values

In [3]:
train_images_original.shape, train_labels_original, test_images_original.shape, test_labels_original.shape

((60000, 28, 28),
 array([5, 0, 4, ..., 5, 6, 8], dtype=uint8),
 (10000, 28, 28),
 (10000,))

Extracting images

In [0]:
train_images_new = (train_images_original.reshape(60000,28 * 28))
train_images_new = train_images_new.astype('float32')/255


test_images_new = (test_images_original.reshape(test_images_original.shape[0],28 * 28))
test_images_new = test_images_new.astype('float32')/255

Converting integer vectors to binary

In [0]:
train_labels_new = (keras.utils.to_categorical(train_labels_original,10))
y_c = test_labels_original
test_labels_new = (keras.utils.to_categorical(test_labels_original,10))

Displaying new values

In [6]:
train_images_new.shape, train_labels_new.shape, test_images_new.shape, test_labels_new.shape

((60000, 784), (60000, 10), (10000, 784), (10000, 10))

Definitions

In [0]:
# Logistic Regression
def log_reg(W,x,b):
  y = np.dot(W,np.transpose(x)) + (b)
  z = sigmoid(y)
  return z

# Sigmoid function
def sigmoid(x): 
  z = 1/(1+np.exp(-x))
  return z

# Derivative of Sigmoid function
def sigmoid_prime(y1):
  return sigmoid(y1)*(1-sigmoid(y1))

# Minibatch SGD - Binary Cross Entropy Loss
def minibatch(W,train_images_new,b,train_labels_new,z,learning_rate,batch_size):
  #loss = np.sum((z - np.transpose(train_labels_new)), axis = 1, keepdims =True)/batch_size
  loss = z - np.transpose(train_labels_new)
  gradient_W = np.dot(loss,train_images_new)
  gradient_b = np.sum(loss,axis=1,keepdims=True)
  
  W = W - learning_rate*gradient_W
  b = b - learning_rate*gradient_b
  return W,b

Initializing the weight vector,  bias, learning rate, batch size and number of epochs

In [0]:
W = np.random.randn(10,784)*0.01
b = np.random.randn(10,1)*0.01

learning_rate = 0.004
batch_size = 10
epochs = 100

Training and determining the classifier with maximum accuracy

In [9]:
for j in range(epochs):
  for i in range(0, train_images_new.shape[0], batch_size):
    z = log_reg(W,train_images_new[i:i+batch_size],b)
    W,b = minibatch(W,train_images_new[i:i+batch_size],b,train_labels_new[i:i+batch_size],z,learning_rate,batch_size)
y_test = log_reg(W,test_images_new,b)
y_test = np.transpose(y_test)
max_list = list()
for i in range(10000):
  max_list.append(np.argmax(y_test[i]))
cnt = 0
for i in range(10000):
  if max_list[i] == y_c[i]:
    cnt += 1
acc = cnt/100
print('Accuracy:', acc)
print('Classifier with highest accuracy is:', np.argmax(max_list))

Accuracy: 91.43
Classifier with highest accuracy is: 7
